# Evaluation

In [1]:
utils_path = "../utils/"
corpus_path = "../datasets/final_dataset_v4_to_publish/"
codes_d_path = "../datasets/final_dataset_v4_to_publish/codiesp_codes/codiesp-D_codes.tsv"
test_gs_path = corpus_path + "test/testX.tsv"

In [2]:
import tensorflow as tf

# Auxiliary components
import sys
sys.path.insert(0, utils_path)
from nlp_utils import *

RES_DIR = "../results/CodiEsp/final_exec/"

TYPE_ANN = "DIAGNOSTICO"
TYPE_TASK = TYPE_ANN[0].lower()

# GS data
df_test_gs = format_codiesp_x_gs(test_gs_path)

valid_codes = set(pd.read_csv(codes_d_path, sep='\t', header=None, 
                                  usecols=[0])[0].tolist())
valid_codes = set([x.lower() for x in valid_codes])

2022-09-15 07:57:16.377553: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4812: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


## Evaluation

In [3]:
def check_ner_norm_performance(model_name, arr_execs):
    """
    Sanity-check procedure that prints the NER-NORM performance of each single model execution.
    """
    for i_exec in arr_execs:
        print("Exec " + str(i_exec) + ":")
        df_test_preds_ner = pd.read_csv(RES_DIR + "df_test_preds_ner_" + TYPE_TASK + "_multi_task_" + \
                str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
        # Adapt to CodiEsp format
        df_test_preds_ner['label_pred'] = TYPE_ANN
        df_test_preds_ner['pos_pred'] = [str(row['start']) + ' ' + str(row['end']) for index, row in df_test_preds_ner.iterrows()]
        df_test_preds_ner['code'] = 'n23' if TYPE_ANN == 'DIAGNOSTICO' else 'bn20'
        df_test_preds_ner = df_test_preds_ner[['clinical_case', 'pos_pred', 'label_pred', 'code']]
        print("NER performance:", calculate_codiesp_ner_metrics(
            df_gs=df_test_gs[df_test_gs['label_gs'] == TYPE_ANN], 
            df_pred=format_codiesp_x_pred_df(
                df_run=df_test_preds_ner,
                valid_codes=valid_codes
            )
        ))
        df_test_preds_norm = pd.read_csv(RES_DIR + "df_test_preds_norm_" + TYPE_TASK + "_multi_task_" + \
                str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
        print("NORM performance:", calculate_codiesp_x_metrics(
            df_gs=df_test_gs[df_test_gs['label_gs'] == TYPE_ANN], 
            df_pred=format_codiesp_x_pred_df(
                df_run=df_test_preds_norm,
                valid_codes=valid_codes
            )
        ), end="\n\n")

In [4]:
def model_performance(dict_names_execs, subtask='norm',
                      round_n=3):
    """
    Generate a pd.DataFrame with the statistics of the performance of each model.
    
    dict_names_seeds: each key is a string with the model name, and 
                      each value is a list with the seeds of the corresponding model.
    """
    res_dict = {}
    for model_name in dict_names_execs:
        p_res, r_res, f1_res = [], [], []
        for i_exec in dict_names_execs[model_name]:
            df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_" + subtask + "_" + TYPE_TASK + "_multi_task_" + \
                    str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
            if subtask == 'ner':
                # Adapt to CodiEsp format
                df_test_preds['label_pred'] = TYPE_ANN
                df_test_preds['pos_pred'] = [str(row['start']) + ' ' + str(row['end']) for index, row in df_test_preds.iterrows()]
                df_test_preds['code'] = 'n23' if TYPE_ANN == 'DIAGNOSTICO' else 'bn20'
                df_test_preds = df_test_preds[['clinical_case', 'pos_pred', 'label_pred', 'code']]
                p, r, f1 = calculate_codiesp_ner_metrics(
                    df_gs=df_test_gs[df_test_gs['label_gs'] == TYPE_ANN], 
                    df_pred=format_codiesp_x_pred_df(
                        df_run=df_test_preds,
                        valid_codes=valid_codes
                    )
                )
            else:
                p, r, f1 = calculate_codiesp_x_metrics(
                    df_gs=df_test_gs[df_test_gs['label_gs'] == TYPE_ANN], 
                    df_pred=format_codiesp_x_pred_df(
                        df_run=df_test_preds,
                        valid_codes=valid_codes
                    )
                )
            p_res.append(p)
            r_res.append(r)
            f1_res.append(f1)
        p_res_stat = pd.Series(p_res).describe()
        r_res_stat = pd.Series(r_res).describe()
        f1_res_stat = pd.Series(f1_res).describe()
        res_dict[model_name] = {"P_avg": round(p_res_stat['mean'], round_n), "P_std": round(p_res_stat['std'], round_n), 
                                "P_max": round(p_res_stat['max'], round_n),
                                "R_avg": round(r_res_stat['mean'], round_n), "R_std": round(r_res_stat['std'], round_n), 
                                "R_max": round(r_res_stat['max'], round_n),
                                "F1_avg": round(f1_res_stat['mean'], round_n), "F1_std": round(f1_res_stat['std'], round_n), 
                                "F1_max": round(f1_res_stat['max'], round_n)}
    return pd.DataFrame(res_dict, index=["P_avg", "P_std", "P_max", 
                                         "R_avg", "R_std", "R_max", 
                                         "F1_avg", "F1_std", "F1_max"]).transpose()    

In [5]:
def format_df_paper(df_res):
    arr_metrics = ["P", "R", "F1"]
    arr_cols = []
    for metric in arr_metrics:
        df_res[metric + '_avg_std'] = df_res.apply(
            lambda x: "." + str(x[metric + '_avg']).split('.')[-1] + " ± " + \
                "." + str(x[metric + '_std']).split('.')[-1], 
            axis=1
        )
        df_res[metric + '_max'] = df_res[metric + '_max'].apply(
            lambda x: "." + str(x).split('.')[-1]
        )
        arr_cols += [metric + '_avg_std', metric + '_max']
    return df_res[arr_cols]

In [6]:
# Sanity check

In [6]:
m_name = "xlmr"
execs = [1, 2, 3, 4, 5]

check_ner_norm_performance(model_name=m_name, arr_execs=execs)

Exec 1:


/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


NER performance: (0.7268, 0.6313, 0.6757)
NORM performance: (0.6342, 0.5496, 0.5889)

Exec 2:


/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


NER performance: (0.7457, 0.6258, 0.6805)
NORM performance: (0.6368, 0.538, 0.5833)

Exec 3:


...

/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


NER performance: (0.745, 0.6387, 0.6878)
NORM performance: (0.6398, 0.5475, 0.5901)

Exec 5:


/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


NER performance: (0.7602, 0.6338, 0.6913)
NORM performance: (0.6461, 0.5486, 0.5933)



/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


In [7]:
m_name = "xlmr_galen"
execs = [1, 2, 3, 4, 5]

check_ner_norm_performance(model_name=m_name, arr_execs=execs)

Exec 1:


/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


NER performance: (0.7851, 0.6327, 0.7007)
NORM performance: (0.6698, 0.5468, 0.6021)

Exec 2:


/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


NER performance: (0.784, 0.6277, 0.6972)
NORM performance: (0.6768, 0.5468, 0.6049)

Exec 3:


...

/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


NER performance: (0.7714, 0.631, 0.6942)
NORM performance: (0.6537, 0.538, 0.5902)

Exec 5:


/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


NER performance: (0.7771, 0.6338, 0.6982)
NORM performance: (0.6537, 0.5394, 0.5911)



/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


## Paper

### NORM

In [8]:
model_performance(
    {
        'beto': [1, 2, 3, 4, 5], 
        'beto_galen': [1, 2, 3, 4, 5],
        'mbert': [1, 2, 3, 4, 5], 
        'mbert_galen': [1, 2, 3, 4, 5],
        'xlmr': [1, 2, 3, 4, 5], 
        'xlmr_galen': [1, 2, 3, 4, 5]
    }, 
    subtask='norm'
)

/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteratio

,P_avg,P_std,P_max,R_avg,R_std,R_max,F1_avg,F1_std,F1_max
beto,0.672,0.018,0.695,0.521,0.014,0.533,0.586,0.008,0.598
beto_galen,0.664,0.021,0.696,0.527,0.005,0.534,0.587,0.010,0.601
mbert,0.661,0.017,0.684,0.540,0.006,0.546,0.595,0.004,0.598
mbert_galen,0.671,0.008,0.681,0.544,0.002,0.546,0.601,0.004,0.605
xlmr,0.643,0.010,0.658,0.546,0.005,0.550,0.591,0.006,0.598
xlmr_galen,0.664,0.010,0.677,0.543,0.004,0.547,0.597,0.007,0.605


In [9]:
format_df_paper(
    model_performance(
        {
            'beto': [1, 2, 3, 4, 5], 
            'beto_galen': [1, 2, 3, 4, 5],
            'mbert': [1, 2, 3, 4, 5], 
            'mbert_galen': [1, 2, 3, 4, 5],
            'xlmr': [1, 2, 3, 4, 5], 
            'xlmr_galen': [1, 2, 3, 4, 5]
        }, 
        subtask='norm'
    )
)

/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteratio

,P_avg_std,P_max,R_avg_std,R_max,F1_avg_std,F1_max
beto,.672 ± .018,.695,.521 ± .014,.533,.586 ± .008,.598
beto_galen,.664 ± .021,.696,.527 ± .005,.534,.587 ± .01,.601
mbert,.661 ± .017,.684,.54 ± .006,.546,.595 ± .004,.598
mbert_galen,.671 ± .008,.681,.544 ± .002,.546,.601 ± .004,.605
xlmr,.643 ± .01,.658,.546 ± .005,.55,.591 ± .006,.598
xlmr_galen,.664 ± .01,.677,.543 ± .004,.547,.597 ± .007,.605


### NER

In [10]:
model_performance(
    {
        'beto': [1, 2, 3, 4, 5], 
        'beto_galen': [1, 2, 3, 4, 5],
        'mbert': [1, 2, 3, 4, 5], 
        'mbert_galen': [1, 2, 3, 4, 5],
        'xlmr': [1, 2, 3, 4, 5], 
        'xlmr_galen': [1, 2, 3, 4, 5]
    }, 
    subtask='ner'
)

/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteratio

,P_avg,P_std,P_max,R_avg,R_std,R_max,F1_avg,F1_std,F1_max
beto,0.797,0.015,0.815,0.607,0.019,0.620,0.688,0.009,0.696
beto_galen,0.787,0.018,0.812,0.620,0.004,0.622,0.693,0.005,0.698
mbert,0.771,0.012,0.786,0.621,0.011,0.633,0.688,0.003,0.691
mbert_galen,0.778,0.008,0.786,0.626,0.005,0.634,0.694,0.004,0.698
xlmr,0.748,0.015,0.763,0.632,0.005,0.639,0.685,0.007,0.691
xlmr_galen,0.779,0.006,0.785,0.632,0.002,0.634,0.698,0.002,0.701


In [11]:
format_df_paper(
    model_performance(
        {
            'beto': [1, 2, 3, 4, 5], 
            'beto_galen': [1, 2, 3, 4, 5],
            'mbert': [1, 2, 3, 4, 5], 
            'mbert_galen': [1, 2, 3, 4, 5],
            'xlmr': [1, 2, 3, 4, 5], 
            'xlmr_galen': [1, 2, 3, 4, 5]
        }, 
        subtask='ner'
    )
)

/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/comun/guillermo/NLP/scripts/../utils/nlp_utils.py:4918: FutureWarning: Columnar iteratio

,P_avg_std,P_max,R_avg_std,R_max,F1_avg_std,F1_max
beto,.797 ± .015,.815,.607 ± .019,.62,.688 ± .009,.696
beto_galen,.787 ± .018,.812,.62 ± .004,.622,.693 ± .005,.698
mbert,.771 ± .012,.786,.621 ± .011,.633,.688 ± .003,.691
mbert_galen,.778 ± .008,.786,.626 ± .005,.634,.694 ± .004,.698
xlmr,.748 ± .015,.763,.632 ± .005,.639,.685 ± .007,.691
xlmr_galen,.779 ± .006,.785,.632 ± .002,.634,.698 ± .002,.701


Save the (F1) performance of all executions of all models

In [12]:
def model_f1_values(dict_names_execs, subtask='norm'):
    """
    Generate a vector containing the F1 performance of all executions of all models, in the given order.
    
    dict_names_execs: each key is a string with the model name, and 
                      each value is a list with the random execs of the corresponding model.
    """
    arr_values = []
    for model_name in dict_names_execs:
        for i_exec in dict_names_execs[model_name]:
            df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_" + subtask + "_" + TYPE_TASK + "_multi_task_" + \
                    str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
            if subtask == 'ner':
                # Adapt to CodiEsp format
                df_test_preds['label_pred'] = TYPE_ANN
                df_test_preds['pos_pred'] = [str(row['start']) + ' ' + str(row['end']) for index, row in df_test_preds.iterrows()]
                df_test_preds['code'] = 'n23' if TYPE_ANN == 'DIAGNOSTICO' else 'bn20'
                df_test_preds = df_test_preds[['clinical_case', 'pos_pred', 'label_pred', 'code']]
                _, _, f1 = calculate_codiesp_ner_metrics(
                    df_gs=df_test_gs[df_test_gs['label_gs'] == TYPE_ANN], 
                    df_pred=format_codiesp_x_pred_df(
                        df_run=df_test_preds,
                        valid_codes=valid_codes
                    )
                )
            else:
                _, _, f1 = calculate_codiesp_x_metrics(
                    df_gs=df_test_gs[df_test_gs['label_gs'] == TYPE_ANN], 
                    df_pred=format_codiesp_x_pred_df(
                        df_run=df_test_preds,
                        valid_codes=valid_codes
                    )
                )
            arr_values.append(f1)
    return arr_values

In [ ]:
# NER

In [6]:
arr_val = model_f1_values(
    {
        'beto': [1, 2, 3, 4, 5], 
        'beto_galen': [1, 2, 3, 4, 5],
        'mbert': [1, 2, 3, 4, 5], 
        'mbert_galen': [1, 2, 3, 4, 5],
        'xlmr': [1, 2, 3, 4, 5], 
        'xlmr_galen': [1, 2, 3, 4, 5]
    }, 
    subtask='ner'
)

/mnt/ext/almacen/comun/guillermo/Plan-TL/CodiEsp/NER/../../nlp_utils_norm_HERE_Text.py:4918: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


In [7]:
pd.DataFrame(arr_val).to_csv(RES_DIR + "ner_f1_exec_" + TYPE_TASK + "_multi_task.csv", index=False, header=False, sep = '\t')

In [ ]:
# NORM

In [8]:
arr_val = model_f1_values(
    {
        'beto': [1, 2, 3, 4, 5], 
        'beto_galen': [1, 2, 3, 4, 5],
        'mbert': [1, 2, 3, 4, 5], 
        'mbert_galen': [1, 2, 3, 4, 5],
        'xlmr': [1, 2, 3, 4, 5], 
        'xlmr_galen': [1, 2, 3, 4, 5]
    }, 
    subtask='norm'
)

In [9]:
pd.DataFrame(arr_val).to_csv(RES_DIR + "norm_f1_exec_" + TYPE_TASK + "_multi_task.csv", index=False, header=False, sep = '\t')